# #300 Warren truss rail bridge
<i>A basic steel truss bridge structure is modelled with beam elements. Track layouts are created for subsequent optimisation of rail loading</i>
***

![Preview](../_img/jupyter_notebook_300.png)

Input Parameters

In [ ]:
truss_height = 5
truss_width = 5
segment_lengths = [5,5,5,5,5]

In [ ]:
import sys; sys.path.append('../') # Reference modules in parent directory
from shared.LPI import *
lusas = get_lusas_modeller()
if lusas.existsDatabase() and lusas.db().isModified():
    raise Exception("This script will create a new model. Please save or close the current model and try again")

# Initialise the helpers module with a reference to LUSAS
import shared.Helpers as Helpers
Helpers.initialise(lusas)

# Create a new model
lusas.newProject("Structural", "Warren_Truss.mdl")
# Get a reference to the current model database for convenience
db = lusas.database() 
# 3D model with Z vertical
db.setAnalysisCategory("3D")
db.setVerticalDir("Z")

# Set the unit system
db.setModelUnits("kN,m,t,s,C")
# Set the current view
lusas.view().setIsometric()
lusas.view().geometry().autoColourByAttributes("Geometric", True)


In [ ]:
# Mesh attribute for all members. 
mesh_attr = db.createMeshLine("Beam mesh 1m").setSize("BMI21", 1)

# Steel material attributes for all members.
steel_mat_attr = db.createIsotropicMaterial("Steel", 2e6, 0.3, 7.4)

# Bottom chord geometric attribute
bot_chrd_geom_attr = db.createGeometricLine("Bottom chord")
bot_chrd_geom_attr.setFromLibrary("Canadian Sections", "W: Wide-Flange Shapes", "W920x253", 0, 0, 0)

# Top chord geometric attribute
top_chrd_geom_attr = db.createGeometricLine("Top chord")
top_chrd_geom_attr.setFromLibrary("Canadian Sections", "W: Wide-Flange Shapes", "W920x253", 0, 0, 0)

# Diagonal geometric attribute
web_geom_attr = db.createGeometricLine("Web diagonals")
web_geom_attr.setFromLibrary("Canadian Sections", "W: Wide-Flange Shapes", "W920x253", 0, 0, 0)

# Bottom chord cross girder geometric attribute
btm_crs_geom_attr = db.createGeometricLine("Bottom transverse girders")
btm_crs_geom_attr.setFromLibrary("Canadian Sections", "W: Wide-Flange Shapes", "W920x253", 0, 0, 0)

# Bottom chord bracing geometric attribute
btm_brc_geom_attr = db.createGeometricLine("Bottom bracing")
btm_brc_geom_attr.setFromLibrary("Canadian Sections", "W: Wide-Flange Shapes", "W530x272", 0, 0, 0)

# Top chord cross girder geometric attribute
top_crs_geom_attr = db.createGeometricLine("Top transverse girders")
top_crs_geom_attr.setFromLibrary("Canadian Sections", "W: Wide-Flange Shapes", "W920x253", 0, 0, 0)

# Top chord bracing geometric attribute
top_brc_geom_attr = db.createGeometricLine("Top bracing")
top_brc_geom_attr.setFromLibrary("Canadian Sections", "W: Wide-Flange Shapes", "W530x272", 0, 0, 0)

# Set the element types
for geo_attr in [bot_chrd_geom_attr, top_chrd_geom_attr, web_geom_attr, btm_crs_geom_attr]:
    geo_attr.setValue("elementType", "3D Thick Beam")


# Support attributes
support_pinned_attr = db.createSupportStructural("Pinned").setStructural("R", "R", "R", "F", "F", "F", "F", "F", "C", "F") # F=Free, R=Restrained
support_slide_attr  = db.createSupportStructural("Slide").setStructural("F", "R", "R", "F", "F", "F", "F", "F", "C", "F")


In [ ]:
# Set the mesh lock to prevent remeshing on each assignment
db.setMeshLock(True)

# Set the mesh and material attributes as default so they are automatically assigned to all new created geometry
db.setAsDefault("Line mesh", mesh_attr)
db.setAsDefault("Isotropic Material", steel_mat_attr)

In [ ]:
# Create a class to hold the points and lines for each truss
class WarrenTruss:

    def __init__(self, originX:float, originY:float, originZ:float):
        self.originX = originX
        self.originY = originY
        self.originZ = originZ
        
        self.bottom_chord_points = []
        self.top_chord_points    = []
        self.bottom_chord_lines  = []
        self.top_chord_lines     = []
        self.web_lines           = []


    def create_bottom_chord_points(self):
        # List of points in the bottom chord, initialised with a point at the origin
        self.bottom_chord_points = [Helpers.create_point(self.originX,self.originY,self.originZ)]

        # Append additional points for the segment lengths
        x = self.originX
        for s in segment_lengths:
            x+=s
            self.bottom_chord_points.append(Helpers.create_point(x, self.originY, self.originZ))


    def create_top_chord_points(self):
        # Offset the top chord point by half the first segment length
        x = self.originX + segment_lengths[0] / 2

        # List of points in the top chord
        self.top_chord_points = [Helpers.create_point(self.originX + x, self.originY, self.originZ + truss_height)]

        # Append additional points for the segment lengths
        for i in range(0, len(segment_lengths)-1):
            x+=segment_lengths[i]
            self.top_chord_points.append(Helpers.create_point(x, self.originY, self.originZ + truss_height))


    def create_bottom_chord_lines(self):
        # Connect the points along the bottom chord
        for i in range(0, len(segment_lengths)):
            line = Helpers.create_line_from_points(self.bottom_chord_points[i], self.bottom_chord_points[i+1])
            self.bottom_chord_lines.append(line)


    def create_top_chord_lines(self):
        # Connect the points along the top chord
        for i in range(0, len(segment_lengths)-1):
            line = Helpers.create_line_from_points(self.top_chord_points[i], self.top_chord_points[i+1])
            self.top_chord_lines.append(line)


    def create_web_lines(self):

        for i in range(0, len(segment_lengths)):
            line = Helpers.create_line_from_points(self.bottom_chord_points[i], self.top_chord_points[i])
            self.web_lines.append(line)

        for i in range(0, len(segment_lengths)):
            line = Helpers.create_line_from_points(self.top_chord_points[i], self.bottom_chord_points[i+1])
            self.web_lines.append(line)

In [ ]:
truss1 = WarrenTruss(0,0,0)
truss1.create_bottom_chord_points()
truss1.create_top_chord_points()

truss2 = WarrenTruss(0, truss_width, 0)
truss2.create_bottom_chord_points()
truss2.create_top_chord_points()

In [ ]:
# # Create the bottom chord lines. Set the default section for automatic assignment to the bottom chord
db.setAsDefault("Line Geometric", bot_chrd_geom_attr)

truss1.create_bottom_chord_lines()
truss2.create_bottom_chord_lines()

In [ ]:
# Create the top chord lines. Set the default section for automatic assignment to the top chord
db.setAsDefault("Line Geometric", top_chrd_geom_attr)

truss1.create_top_chord_lines()
truss2.create_top_chord_lines()

In [ ]:
# Create web lines. Set the default section for automatic assignment
db.setAsDefault("Line Geometric", web_geom_attr)

truss1.create_web_lines()
truss2.create_web_lines()

In [ ]:
# Create bottom cross girder lines between the two truss section. Set the default section for automatic assignment
db.setAsDefault("Line Geometric", btm_crs_geom_attr)

for i in range(0, len(segment_lengths)+1):
    Helpers.create_line_from_points(truss1.bottom_chord_points[i], truss2.bottom_chord_points[i])

In [ ]:
# Create bottom cross girder lines between the two truss section. Set the default section for automatic assignment
db.setAsDefault("Line Geometric", btm_brc_geom_attr)

for i in range(0, len(segment_lengths)):
    if i%2==0:
        Helpers.create_line_from_points(truss1.bottom_chord_points[i], truss2.bottom_chord_points[i+1])
    else:
        Helpers.create_line_from_points(truss2.bottom_chord_points[i], truss1.bottom_chord_points[i+1])

In [ ]:
# Create top cross girder lines between the two truss section. Set the default section for automatic assignment
db.setAsDefault("Line Geometric", top_crs_geom_attr)

for i in range(0, len(segment_lengths)):
    Helpers.create_line_from_points(truss1.top_chord_points[i], truss2.top_chord_points[i])

In [ ]:
# Create top cross girder lines between the two truss section. Set the default section for automatic assignment
db.setAsDefault("Line Geometric", top_brc_geom_attr)

for i in range(0, len(segment_lengths)-1):
    if i%2==0:
        Helpers.create_line_from_points(truss1.top_chord_points[i], truss2.top_chord_points[i+1])
    else:
        Helpers.create_line_from_points(truss2.top_chord_points[i], truss1.top_chord_points[i+1])

Assign supports:

(if you get a "Can't visualise attributes without a mesh" warning, just click OK. This can be avoided by assigning a mesh first.)

In [ ]:
# Assign supports
support_pinned_attr.assignTo(truss1.bottom_chord_points[0])
support_pinned_attr.assignTo(truss2.bottom_chord_points[0])
support_slide_attr.assignTo(truss1.bottom_chord_points[-1])
support_slide_attr.assignTo(truss2.bottom_chord_points[-1])

# Create loads

In [ ]:
# Get the automatically created loadcase in analysis 1 and add automatic gravity to it
gravity_loadcase : IFLoadcase = db.getLoadset("Loadcase 1", 0)
gravity_loadcase.setName("Gravity")
gravity_loadcase.addGravity(True)

In [ ]:
# Create a loadcase for wind loading
loadset = db.createLoadcase("Wind in Y", "Analysis 1")

# Create a projected pressure load for the wind in the Y direction
wind_load_attr = db.createLoadingBeamProjectedPressure("Wind load Y")
wind_load_attr.setLoadDirection("globalY")
wind_load_attr.setLoading("10.0")
wind_load_attr.setLoadingWidth("factor", 1.0)

# Create the assignment settings
assignment = lusas.assignment().setAllDefaults().setLoadset(loadset)

# Assign to truss 1 lines
wind_load_attr.assignTo(truss1.top_chord_lines, assignment)
wind_load_attr.assignTo(truss1.web_lines, assignment)
wind_load_attr.assignTo(truss1.bottom_chord_lines, assignment)

In [ ]:
# Create a search area attribute to identify the members to be loaded by the rail load
search_area_attr = db.createSearchArea("Deck")

# Create an object set containing all the lines with the bottom cross girder geometric assignment
# The search area needs to form an enclosed area so it contains the bottom chords and cross girders
deck_lines = lusas.newObjectSet()
deck_lines.add(btm_crs_geom_attr)
deck_lines.add(bot_chrd_geom_attr)

# Assign the search area to them
search_area_attr.assignTo(deck_lines, lusas.assignment().setAllDefaults())

In [ ]:
# Create a single track definition 
rail_track = db.createRailTrackDefinition("TrackDefn1")
track_centreline = rail_track.getCentrelinePath()
track_centreline.getDefn().deleteAllLines()
track_centreline.getDefn().addStraightV(-10.0, truss_width/2, 0.0, sum(segment_lengths)+10, truss_width/2, 0.0)
track_centreline.getDefn().setFacetData(20, 6, -1.0, 3.0, -1.0)
track_centreline.getDefn().setSmoothing(False)
track_centreline.getDefn().setFilletType(0, True)
rail_track.setWidth(1.495)
rail_track.closeEnd(False)
rail_track.closeStart(False)

# And add to the overall track layout arrangement
railTrackLayout0 = db.createRailTrackLayout("TrackLayout1").addTrackDefinition(rail_track)

In [ ]:
# Create the mesh before generating the DMI Influence analysis
db.setMeshLock(False)
db.resetMesh()
db.updateMesh()


In [ ]:
# Create the rail Influence analysis
rail_dmi_analysis = db.createAnalysisRailDMI("Rail DMI Analysis 1")
rail_dmi_analysis.setLoadDirection("Vertical")
rail_dmi_analysis.setLoadMagnitude(1.0E3)
rail_dmi_analysis.setSearchArea("Deck")
rail_dmi_analysis.setSearchAssignType("area")
rail_dmi_analysis.setIsGrillageType(2)
rail_dmi_analysis.setRailTrackLayouts(["TrackLayout1"])
rail_dmi_analysis.setSelectedResultsGroup("assignments")
rail_dmi_analysis.setSelectedElementOutputGroup("all")
rail_dmi_analysis.setSelectedNodeOutputGroup("all")

In [ ]:
# Create the influence envelope attribute for member axial force
inf_attr = db.createInfluenceEnvelope("Axial force influence envelope")
inf_attr.setResultsTransformElement()
inf_attr.setEntity("Force/Moment - Thick 3D Beam")
inf_attr.addComponentNames(["Fx"])
inf_attr.includeCoincident(True)

# Create the assignment settings
assignment = lusas.assignment().setAllDefaults().setAnalysis("Rail DMI Analysis 1")

# Assign the influence envelope attribute to the analysis
inf_attr.assignTo(truss1.top_chord_lines, assignment)
inf_attr.assignTo(truss1.web_lines, assignment)
inf_attr.assignTo(truss1.bottom_chord_lines, assignment)

# Solve Analyses

In [ ]:
# Solve the analysis 
db.getAnalysis("Analysis 1").solve(True)
rail_dmi_analysis.solve(True)
db.openAllResults(True, False)

Currently the VLO Envelope Analysis must be defined through the user interface